## Neural Networks: 

from https://pad.gwdg.de/s/Machine_Learning_For_Physicists_2021#

- Optimized gradient descent algorithms 
- Recurrent Neural Networks

## How to speed-up stochastic gradient descent?

- Accelerate ("momentum") towards minimum
- Automatically choose learning rate
- Different rates for different weights

**Few gradient update methods:** 

*Ref: An overview of gradient descent optimization algorithms - Sebastian Ruder (https://arxiv.org/abs/1609.04747)*

    1. adagrad - divide by RMS of all previous gradients
    2. RMSprop - divide by RMS of last few previous gradients 
    3. adadelta - same, but multiply also by RMS of last few parameters updates
    4. ADAM - divide running avg of last few gradients by RMS of last few gradients (with corrections during earliest steps)


### 1. adagrad: (adaptive gradient)

Gradient: $\theta_j$ one of the many parameters (weights) of the net; $g_j$ j-component of the gradient $g$.
\begin{equation}
g_j=\frac{\partial C}{\partial \theta_j} 
\end{equation}

Standard: I change the vector of parameters $\theta$ by taking the *old* vector and subtracting an amount of $\eta$ (learning rate) times the *old* gradient. 
\begin{equation}
\frac{d\theta}{dt}=-\eta g \qquad \qquad discrete \quad steps: \qquad \theta^{(t+1)}=\theta^{(t)}-\eta g^{(t)}
\end{equation}

We don't know how large/small will be the gradient -> treat the gradients independently. 

Idea: rescale according to estimate of "typical size" of $g$; to do so, square the gradient and sum over all the previous time-steps to get an estimate of how large it is and RMS -$\epsilon$ to avoid division by zero-.
\begin{equation}
\Delta \theta_j=-\eta \cdot \frac{g_j^{(t)}}{\sqrt{\sum_{t'\leq t}(g_j^{(t)})^2+\epsilon}} 
\end{equation}

### 2. RMSprop: 

Problem: the sum of $g$ in the *adagrad* denominator will grow over time -> **decay of learning rate**. 

Solution: Only take RMS of the last few gradients. 

\begin{equation}
V^{(t)}=\gamma V^{(t-1)}+(1-\gamma)(g^{(t)})^2 \qquad \rightarrow \qquad  V^{(t)}=(1-\gamma)\sum_{t'\leq t}\gamma^{t-t'}(g^{(t')})^2
\end{equation}

$V$ = estimate of the variance, being $\gamma$ a decay term (e.g., $\approx 0.9$). A way to accumulate the average but also fade away the old: the influence of the old estimates fades away ($\gamma V^{(t-1)})$. 

>Contributions from $\approx 1/(1-\gamma)$ last terms e.g., for time-independent $g^{(t)}=g$:

\begin{equation}
\sum_{t'\leq t}\gamma^{t-t'}\approx \frac{1}{1-\gamma} \qquad \qquad V^{(t)}=(1-\gamma)\sum_{t'\leq t}\gamma^{t-t'}g^{2}= g^2
\end{equation}

>Update:

\begin{equation}
\Delta \theta_j^{(t)}=-\eta \cdot \frac{g_j^{(t)}}{\sqrt{V_j^{(t)}+\epsilon}} 
\end{equation}

### 3. adadelta: 

Multiply also by RMS of last few parameter updates: 

\begin{equation}
\hat{V}^{(t)}=\gamma \hat{V}^{(t-1)}+(1-\gamma)(\Delta \theta^{(t)})^2
\end{equation}

No learning rate $\eta$
\begin{equation}
\Delta \theta^{(t)}=-\frac{\hat{V}^{(t-1)}}{\hat{V}^{(t)}}g^{(t)}
\end{equation}

### 4. adam: 

Divide running average of last few gradients by RMS of last few gradients (with corrections during earliest steps):

\begin{equation}
n^{(t)}=\beta m^{(t-1)}+ (1-\beta)g^{(t)}
\end{equation}

being $g^{(t)}$ the actual value of the gradient, $\beta<1$, $m^{(t-1)}$ old value of the *mean* gradient, then $n^{(t)}$ isn't the present value of the gradient but something like an avg. over the last few $g's$. 

\begin{equation}
\hat{V}^{(t)}=\gamma \hat{V}^{(t-1)}+(1-\gamma)(\Delta \theta^{(t)})^2
\end{equation}

Little problem: $m^{(t)}\approx 0$,  $V^{(t)}\approx 0$ in first steps. Correct via:

\begin{equation}
\hat{m}^{(t)}=\frac{m^{(t)}}{1-\beta^t}\qquad \qquad \hat{V}^{(t)}=\frac{V^{(t)}}{1-\gamma^t}
\end{equation}

>Step:
\begin{equation}
\Delta \theta^{(t)}=-\eta \cdot \frac{\hat{m}^{(t)}}{\sqrt{\hat{V}^{(t)}+\epsilon}} \qquad \qquad \beta\approx 0.9, \quad \gamma\approx 0.999, \quad \epsilon\approx 10^{-8}
\end{equation}

## Recurrent neural networks:

- Networks with "memory". 
- Useful for: 
            
        Analyzing time-evolution (time-series of data)
        Analyzing and translating sentences
        For control/feedback (robotics or games)
        ...

*Use a convolutional NN: so that we input a time-series (filter size = memory time) and it outputs a modifyed time series. Are OK for problems in local time / short memory.*
       
       But long memories with CNN are challenging: need larger filter sizes, may need to know required memory time 
       beforehand, can expand memory time efficiently by multi-layer network with subsampling (pooling), 
       but this is problematic for precise long-term memory.  
       
**Recurrent neural networks (RNN)**:

(1) Unfolding the inputs in time: each circle is represented in a certain time-step. 

(2) Outputs at given time not only depend on their input at same time. 

(3) Outputs aren't transmitted to the outside but we keep part of those to be available in the next time-step -> things like, for a given circle $i$, taking both the input ($i$) and the previous output ($i-1$) in order to calculate the output ($i$) of the circle (which can be a neuron or a layer).  

(4) Weights (strenght of the arrows) are *time independent*!

       output:  o -> o -> o -> o - (keep memory) -> o -> o -> o -> ...
                ^    ^    ^    ^                    ^    ^    ^ 
                |    |    |    |                    |    |    |
       input:   o    o    o    o                    o    o    o 
       --------------------------------------------------------------> time
        
>Advantage: in principle this could give arbitrarily long memory. Each circle $o$ may represent multiple neurons (i.e., layer). Each arrow represents all possible connections between those neurons. 

        - Train on many input sequences 
        - Tell the net what would have been the correct output for this particular sequence
        - Try to adapt my weights in order to make this happen

Training process:

       output:  o -> o -> o -> o - (keep memory) -> o -> o -> o    <- Correct answer is known here; take the gradient of
                ^    ^    ^    ^                    ^    ^    ^       the cost function with respect to this last output 
                |    |    |    |                    |    |    |       but then we get gradients with everything that is 
       hidden:  o -> o -> o -> o -     ...       -> o -> o -> o       connected to this output; need to go backward in time:
                ^    ^    ^    ^                    ^    ^    ^       BACKPROPAGATION GOES DOWN IN THE LAYERS BUT ALSO 
                |    |    |    |                    |    |    |       BACK IN TIME
       input:   o    o    o    o       ...          o    o    o 
       ---------------------------------------------------------> time
                                                                <----  BACKPROPAGATION THROUGH TIME
                                                                    |
                                                                    v
      
>**Challenge**: long memories with RNN are challenging due to a feature of backpropagation -> "*Exploding gradients*" / "*Vanishing gradients*" -> Backpropagation through many layers (deep net) or many time-steps (RNN): 

Each step in the backpropagation is a matrix-vector multiplication, deviation vector may be something like:

\begin{equation}
\Delta_{t-1}=M_t\Delta_t
\end{equation}

Depending on the typical eigenvalues of the matrices $M$, $|\Delta|$ can explode (regions where eigenvalues are large and so the deviation vector keeps growing) or vanish (regions where eigenvalues are small and so the deviation vector vanishes).

**Solution: Long short-term memory (LSTM)**

    Long-term memory = weights that are adapted during training and stored forever
    Short-term memory = input-dependent memory
    
*LSTM tries to have long memory times in a robust way for this short-time memory*: Determine read/write/delete operations of a memory cell via the network (through other neurons). Most of the time, a memory neuron just sits there and isn't used or changed.

**A. Deleting the memory - LSTM: Forget date**
  
                               c_(t-1)          c_t
    memory cell content, c:    o -------x-----> o
    time-step from t-1 to t             |   
                                    keep:  c_t = 1 x c_(t-1)
                                  delete:  c_t = 0 x c_(t-1)
                                  
Multiply the input $c_{t-1}$ by 1 (or 0) if we want to keep it (or delete it). We want these 1 or 0 to be calculated based on the values of some other neurons. 

                               c_(t-1)          c_t
    memory cell content, c:    o -------x-----> o
    time-step from t-1 to t             ^
                                        |   
                                 (forget gate, f)
                                        |
                           input, x_t   o 
                           
Calculate the *forget gate*: value $f$ which may be 1, or 0 or something in-between. 

\begin{equation}
f=\sigma(W^{(f)}x_t+b^{(f)})
\end{equation}

where $\sigma$ is **sigmoid**, $x$, $b$ and $f$ are vectors and $W$ the weight matrix. Now obtain new memory content: **we are multiplying neuron values**

\begin{equation}
c_t=f \star c_{t-1}, \qquad \star=elementwise\quad product
\end{equation}

Product rule for time $t$: the multiplication $\star$ splits the error backpropagation into two branches. 

\begin{equation}
\frac{\partial f_j c_{t-1,j}}{\partial w_\star} = \frac{\partial f_j}{\partial w_\star}c_{t-1,j} +  f_j\frac{\partial c_{t-1,j}}{\partial w_\star}
\end{equation}

>Whole picture: 

                               c_(t-2)          c_(t-1)          c_(t)           c_(t+1)
    memory cell content, c:      o -------x-----> o -------x-----> o -------x-----> o 
    time-step from t-1 to t+1             ^                ^                ^ 
                                          |                |                |
                                   (forget gate, f) (forget gate, f) (forget gate, f)
                                          |                |                |
                        input:  x_(t-1)   o          x_t   o       x_(t+1)  o

**B. Writing new memory - LSTM**

                               c_(t-1)            c_t
    memory cell content, c:    o ------- + -----> o
                                         ^
                                         |
                                         x <-- new value, c'_t
                                         ^          |
                                         |          |
                                 (input gate, i)    |
                                         |          |
                           input, x_t    o ---------|

>$i=\sigma(W^{(i)}x_t+b^{(i)})$ and $c'_t=tanh(W^{(c)}x_t+b^{(c)})$;

**Both delete & write together**: $c_t=f*c_{t-1} + i*c'_t$

**C. Read (output) memory value - LSTM**
               
                                                  ^ 
                                                  | h_t
                                                  x <-----|
                               c_(t-1)            |       |
    memory cell content, c:    o ------- + -----> o  c_t  |
                                                          |
                                                          |
                                                  (output gate, o)
                                                          ^
                                                          |
                                            input, x_t    o

>$o=\sigma(W^{(o)}x_t+b^{(i)})$ and $h_t=o*tanh(c_t)$

**D. LSTM: exploit previous memory output 'h'**

Make $f$, $i$, $o$, etc. at time $t$ depend on output 'h' calculated in previous time step. Otherwise, 'h' could only be used in higher layers, but not to control memory access in present layer

\begin{equation}
f=\sigma(W^{(f)}x_t+U^{(f)}h_{t-1}+b^{(f)})
\end{equation}

likewise for every other quantity. Thus, result of readout can actually influence subsequent operations. Sometimes 'o' is even made to depend on $c_t$. 

Coming back to backpropagation: during those 'silent' time-intervals, there's NO explosion or vanishing gradient. 

\begin{equation}
c_t=c_{t-1}=c_{t-2}=... \qquad \qquad \frac{\partial c_t}{\partial w_\star}=\frac{\partial c_{t-1}}{\partial w_\star}=\frac{\partial c_{t-2}}{\partial w_\star}=...
\end{equation}

In [ ]:
# Adding an LSTM layer with 10 memory cells: each of those cells has the full structure, 
# with f, i, o gates and the memory content c, and the output h. 
rnn.add(LSTM(10, return_sequences=True)) # Whether to return the full-time sequence of outputs or only the final time.

Full application: take an input of 3 neuron values for each time step and producing a time sequence with 2 neuron values for each time step.

                            LSTM ( o o ) 2, output 
                                    ^
                                    |
                         LSTM ( o o o o o ) 5, hidden
                                    ^
                                    |      
                                ( o o o ) 3, input

In [ ]:
def init_memory_net():
    global rnn, batchsize, timesteps
    rnn = Sequential()
    
    rnn.add(LSTM(5, batch_input_shape=(None, timesteps, 3), return_sequences=True))
    rnn.add(LSTM(2, return_sequences=True))
    rnn.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# (!!) batch_input_shape is (batchsize, timesteps, data_dim) -> batch_input_shape=(None, timesteps, 3). 

**Recurrent neural networks (RNN): Character generation**

Timetraces is a sequence of characters; given a certain piece of text, can you predict what's the likelihood to have one or the other letter as the next letter of the sequence. 

Input sequence (each letter in one-hot encoder - one letter = one input neuron):
        
        T H E _ T H E O R Y _ O F _ G E 
        ------------------------------> time

Desired output: predict next character:

        H E _ T H E O R Y _ O F _ G E N
        ------------------------------> time

Network will output probability for **each** possible character at each time step. 

                       input  = one-hot 
                       output = probability distirbution

**Recurrent neural networks (RNN): Word vectors**

Simple one-hot encoding of words needs large vectors; dimension = number of words in dictionary. 

    Input neurons: each of them corresponds to one single word. 
    
    Compress the words in vectors -> and place the vectors in a high-dim. space in such a way that there's a relation 
    between them (e.g., word2vec -> "warm", "cold" and "hot" closer and far from "tree"). 
    
Noise-contrastive estimation: provide few noisy (wrong) examples and train the model to predict that they are fake but that the true one is correct. 

    (1) Give the context words and make it guess the word - 'continuous bag of words'
    (2) Give the word and make it guess the context words - 'skip-gram'

Model tries to predict:

$P_\theta(w,h)$, probability that $w$ is the correct word given the context word $h$, $\theta$ are the parameters of the model (weights, biases and entries of embedding vectors). 

$P_\theta(w,h)=\sigma(W_{jk}e_{k}(h)+b_j)$, $j$, index for word w in dictionart, $k$ index in embedding vector (Einstein sum), $e(h)$ embedding vector for word h, $W,b$ weights and biases. 

At each time-step: go down the gradient of $C=-(lnP_\theta(w_t,h)+\sum_{\hat{w}}ln(1-P_\theta(\hat{w},h)))$, being $\hat{w}$ noisy examples.

In [ ]:
# Layer for mapping word indices to word vectors, for input sequences of some given length

embedding_layer=Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)

# Helper routines @ Keras: 
# Tokenizer (Text preprocessing)
# pad_sequences (Sequences preprocessing)